In [1]:
#  Callbacks preparation

In [2]:
import os

In [3]:
# Go to the src directory
os.chdir("C:\\Users\\Ronan\\Documents\\ML\\Image_Classification\\Picture_Classification_ML_Project\\src")

In [4]:
%pwd

'C:\\Users\\Ronan\\Documents\\ML\\Image_Classification\\Picture_Classification_ML_Project\\src'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from CNN_Classification_Task.constants import *
from CNN_Classification_Task.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
   
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
   
    @property
    # Create the tensorboard callbacks
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)   

    @property
    # Create the checkpoint callbacks for the best epoch
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    # Create the callbacks
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [11]:
try:
    # Load the configuration file
    config = ConfigurationManager()
    # Load the parameters file
    prepare_callbacks_config = config.get_prepare_callback_config()
    # Create the callbacks
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    # Get the callbacks
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-05-19 17:27:39,519: INFO: common: yaml file: C:\Users\Ronan\Documents\ML\Image_Classification\Picture_Classification_ML_Project\config\config.yaml loaded successfully]
[2024-05-19 17:27:39,521: INFO: common: yaml file: C:\Users\Ronan\Documents\ML\Image_Classification\Picture_Classification_ML_Project\params.yaml loaded successfully]
[2024-05-19 17:27:39,521: INFO: common: created directory at: artifacts]
[2024-05-19 17:27:39,522: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-05-19 17:27:39,523: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


: 